In [50]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.naive_bayes import MultinomialNB
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from firebase import firebase

firebase = firebase.FirebaseApplication('https://my-symptom-analyzer-c0179.firebaseio.com/', None)

data = pd.read_csv('data_pivoted.csv')
test_data = pd.read_csv('test.csv')

print(data.shape)
data = data.fillna(0)
test_data = test_data.fillna(0)
#test_data.head()


cols = data.columns.tolist()
cols.remove('disease')
x = data[cols]
xx=test_data[cols]
y = data.disease

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

mnb = MultinomialNB()
mnb = mnb.fit(x_train, y_train)

mnb.score(x_test, y_test)

symptoms =list(xx.columns)
import csv   
#print(symptoms)
i=int(0)
while(True):
    change= firebase.get('/name/','change')
    if(change ==1 ):
        change= firebase.get('/name/','symptoms')
        #print(change)
        patient_symptoms = change
        p_s =[]
        for i in symptoms:
            if(i in patient_symptoms):
                #print("hello ",i)
                p_s.append(1)
            else:
                p_s.append(0)

        with open('test.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow(p_s)

        test_data = pd.read_csv('test.csv')
        test_data = test_data.fillna(0)
        xx=test_data[cols]
        #test_data.head()

        mnb_tot = MultinomialNB()
        mnb_tot = mnb_tot.fit(x,y)

        mnb_tot.score(x_test, y_test)

        disease_pred = mnb_tot.predict(xx)
        print(disease_pred)
        firebase.put('/name/', 'disease', disease_pred[-1])
        firebase.put('/name/', 'change', 0)


(65, 317)
['hernia hiatal' 'tricuspid valve insufficiency' 'decubitus ulcer' 'ileus'
 'ileus' 'ileus' 'biliary calculus' 'ileus' 'fibroid tumor'
 'coronary arteriosclerosis' "Alzheimer's disease" 'schizophrenia' 'ileus'
 'coronary arteriosclerosis' 'ileus' 'fibroid tumor' 'fibroid tumor'
 "Alzheimer's disease" 'fibroid tumor' 'fibroid tumor' 'ileus' 'ileus'
 'fibroid tumor' 'fibroid tumor' 'fibroid tumor' 'ileus' 'fibroid tumor'
 'fibroid tumor' 'ileus' 'biliary calculus' 'ileus' 'encephalopathy'
 'ileus' 'ileus']


KeyboardInterrupt: 

In [17]:

firebase.put('/Chat/list/', 'Disease', '')